<a href="https://colab.research.google.com/github/Nico412-cyber/msci151/blob/main/COIL_Project_Group11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "code",
      "source": [
        "# ============================================\n",
        "# FINAL COMPREHENSIVE FX RETURN ANALYSIS\n",
        "# Random Forest + Bagged Decision Tree + XAI (SHAP)\n",
        "# ============================================\n",
        "import pandas as pd\n",
        "import numpy as np\n",
        "import matplotlib.pyplot as plt\n",
        "import seaborn as sns\n",
        "import warnings\n",
        "warnings.filterwarnings(\"ignore\")\n",
        "from sklearn.model_selection import TimeSeriesSplit, GridSearchCV\n",
        "from sklearn.preprocessing import StandardScaler\n",
        "from sklearn.ensemble import RandomForestRegressor, BaggingRegressor\n",
        "from sklearn.tree import DecisionTreeRegressor\n",
        "from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score\n",
        "from sklearn.inspection import permutation_importance\n",
        "from statsmodels.stats.outliers_influence import variance_inflation_factor\n",
        "import shap\n",
        "\n",
        "# --------------------------\n",
        "# Helper functions\n",
        "# --------------------------\n",
        "def evaluate(y_true, y_pred):\n",
        "    return {\n",
        "        \"RMSE\": np.sqrt(mean_squared_error(y_true, y_pred)),\n",
        "        \"MAE\": mean_absolute_error(y_true, y_pred),\n",
        "        \"R2\": r2_score(y_true, y_pred)\n",
        "    }\n",
        "\n",
        "def print_metrics(name, metrics):\n",
        "    print(f\"{name}: RMSE={metrics['RMSE']:.6f}, MAE={metrics['MAE']:.6f}, R2={metrics['R2']:.4f}\")\n",
        "\n",
        "# --------------------------\n",
        "# 1. Load & Clean Data\n",
        "# --------------------------\n",
        "DATA_PATH = \"/content/final_dataset.xlsx\"\n",
        "df = pd.read_excel(DATA_PATH)\n",
        "df = df.loc[:, ~df.columns.str.contains('^Unnamed')]\n",
        "df['Date'] = pd.to_datetime(df['Date'])\n",
        "df = df.sort_values('Date').reset_index(drop=True)\n",
        "df = df[(df['Date'] >= '2021-01-01') & (df['Date'] <= '2024-06-30')].reset_index(drop=True)\n",
        "\n",
        "print(\"✅ Loaded dataset:\", df.shape)\n",
        "print(\"Date range:\", df['Date'].min(), \"->\", df['Date'].max())\n",
        "\n",
        "# --------------------------\n",
        "# 2. DESCRIPTIVE ANALYSIS\n",
        "# --------------------------\n",
        "print(\"\\n✅ Missing values per column:\")\n",
        "print(df.isnull().sum())\n",
        "\n",
        "print(\"\\n✅ Summary Statistics:\")\n",
        "display(df.describe().T)\n",
        "\n",
        "# Exchange rate trend\n",
        "plt.figure(figsize=(10,4))\n",
        "plt.plot(df['Date'], df['GBPAAUD_ExchangeRate'], label='GBP/AUD Exchange Rate', color='steelblue')\n",
        "plt.title(\"GBP/AUD Exchange Rate Over Time\")\n",
        "plt.xlabel(\"Date\"); plt.ylabel(\"Exchange Rate\"); plt.legend()\n",
        "plt.tight_layout(); plt.show()\n",
        "\n",
        "\n",
        "# Plot for other variables\n",
        "variables = ['Gold_Price', 'Oil_Price', 'GEPU_current', 'UK']\n",
        "for col in variables:\n",
        "    plt.figure(figsize=(12,5))\n",
        "    plt.plot(df['Date'], df[col], label=col)\n",
        "    plt.title(f'{col} Over Time')\n",
        "    plt.xlabel('Date')\n",
        "    plt.ylabel(col)\n",
        "    plt.legend()\n",
        "    plt.show()\n",
        "# --------------------------\n",
        "# 3. Feature Engineering\n",
        "# --------------------------\n",
        "df['GBPAAUD_Daily_Return'] = df['GBPAAUD_ExchangeRate'].pct_change()\n",
        "df['Gold_Price_Daily_Return'] = df['Gold_Price'].pct_change()\n",
        "df['Oil_Price_Daily_Return'] = df['Oil_Price'].pct_change()\n",
        "\n",
        "# Lags & rolling features\n",
        "for lag in [1,2,3]:\n",
        "    df[f'ExchangeRate_Return_Lag{lag}'] = df['GBPAAUD_Daily_Return'].shift(lag)\n",
        "\n",
        "for w in [3,5,10]:\n",
        "    df[f'ExchangeRate_MA{w}'] = df['GBPAAUD_Daily_Return'].rolling(w).mean()\n",
        "    df[f'ExchangeRate_Vol{w}'] = df['GBPAAUD_Daily_Return'].rolling(w).std()\n",
        "\n",
        "# Drop NaN rows after rolling/pct_change\n",
        "df = df.dropna().reset_index(drop=True)\n",
        "print(\"\\n✅ After feature engineering:\", df.shape)\n",
        "\n",
        "\n",
        "# --------------------------\n",
        "# 5. Prepare Train/Test Data\n",
        "# --------------------------\n",
        "target = 'GBPAAUD_Daily_Return'\n",
        "exclude = ['Date','GBPAAUD_ExchangeRate','Gold_Price','Oil_Price','GEPU_current', target]\n",
        "features = [c for c in df.columns if c not in exclude]\n",
        "\n",
        "split_idx = int(len(df) * 0.8)\n",
        "train_df, test_df = df.iloc[:split_idx], df.iloc[split_idx:]\n",
        "X_train, y_train = train_df[features], train_df[target]\n",
        "X_test, y_test = test_df[features], test_df[target]\n",
        "\n",
        "print(f\"\\n✅ Train samples: {len(X_train)}, Test samples: {len(X_test)}\")\n",
        "\n",
        "# --------------------------\n",
        "# 5.5 TIME SERIES CROSS-VALIDATION\n",
        "# --------------------------\n",
        "print(\"\\n✅ Performing Time Series Cross-Validation...\")\n",
        "\n",
        "# Create time series splits (5 folds, respecting temporal order)\n",
        "tscv = TimeSeriesSplit(n_splits=5)\n",
        "\n",
        "# Store CV scores for both models\n",
        "rf_cv_scores = []\n",
        "bdt_cv_scores = []\n",
        "\n",
        "print(\"Cross-validation progress:\")\n",
        "for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train), 1):\n",
        "    # Split data for this fold\n",
        "    X_cv_train, X_cv_val = X_train.iloc[train_idx], X_train.iloc[val_idx]\n",
        "    y_cv_train, y_cv_val = y_train.iloc[train_idx], y_train.iloc[val_idx]\n",
        "\n",
        "    # Scale data for this fold\n",
        "    scaler_cv = StandardScaler()\n",
        "    X_cv_train_scaled = scaler_cv.fit_transform(X_cv_train)\n",
        "    X_cv_val_scaled = scaler_cv.transform(X_cv_val)\n",
        "\n",
        "    # Train Random Forest\n",
        "    rf_cv = RandomForestRegressor(\n",
        "    n_estimators=800,\n",
        "    max_depth=None,         # unlimited depth\n",
        "    min_samples_split=2,\n",
        "    min_samples_leaf=1,\n",
        "    max_features=0.80,\n",
        "    bootstrap=True,\n",
        "    max_samples=1.0,\n",
        "    random_state=42,\n",
        "    n_jobs=-1\n",
        ")\n",
        "    rf_cv.fit(X_cv_train_scaled, y_cv_train)\n",
        "    rf_score = r2_score(y_cv_val, rf_cv.predict(X_cv_val_scaled))\n",
        "    rf_cv_scores.append(rf_score)\n",
        "\n",
        "    # Train Bagged Decision Tree\n",
        "    bdt_cv = BaggingRegressor(\n",
        "    estimator=DecisionTreeRegressor(\n",
        "        max_depth=15,             # let trees grow deeper\n",
        "        min_samples_split=3,\n",
        "        min_samples_leaf=1,\n",
        "        random_state=42\n",
        "    ),\n",
        "    n_estimators=800,             # more trees = better averaging\n",
        "    max_samples=0.9,              # use more samples\n",
        "    max_features=1.0,             # allow more features → stronger trees\n",
        "    bootstrap=True,\n",
        "    random_state=42,\n",
        "    n_jobs=-1\n",
        ")\n",
        "    bdt_cv.fit(X_cv_train_scaled, y_cv_train)\n",
        "    bdt_score = r2_score(y_cv_val, bdt_cv.predict(X_cv_val_scaled))\n",
        "    bdt_cv_scores.append(bdt_score)\n",
        "\n",
        "    print(f\"  Fold {fold}: RF R²={rf_score:.4f}, Bagged DT R²={bdt_score:.4f}\")\n",
        "\n",
        "# Print CV summary\n",
        "print(\"\\n✅ Cross-Validation Results:\")\n",
        "print(f\"Random Forest     - Mean R²: {np.mean(rf_cv_scores):.4f} (+/- {np.std(rf_cv_scores):.4f})\")\n",
        "print(f\"Bagged Dec. Tree  - Mean R²: {np.mean(bdt_cv_scores):.4f} (+/- {np.std(bdt_cv_scores):.4f})\")\n",
        "\n",
        "# Visualization of CV scores\n",
        "plt.figure(figsize=(10,5))\n",
        "folds = list(range(1, 6))\n",
        "plt.plot(folds, rf_cv_scores, marker='o', label='Random Forest', linewidth=2)\n",
        "plt.plot(folds, bdt_cv_scores, marker='s', label='Bagged Decision Tree', linewidth=2, color='orange')\n",
        "plt.axhline(np.mean(rf_cv_scores), linestyle='--', color='steelblue', alpha=0.5, label=f'RF Mean: {np.mean(rf_cv_scores):.3f}')\n",
        "plt.axhline(np.mean(bdt_cv_scores), linestyle='--', color='orange', alpha=0.5, label=f'BDT Mean: {np.mean(bdt_cv_scores):.3f}')\n",
        "plt.xlabel('Fold')\n",
        "plt.ylabel('R² Score')\n",
        "plt.title('Time Series Cross-Validation Performance')\n",
        "plt.legend()\n",
        "plt.grid(True, alpha=0.3)\n",
        "plt.tight_layout()\n",
        "plt.show()\n",
        "\n",
        "# --------------------------\n",
        "# 6. VIF Pruning\n",
        "# --------------------------\n",
        "def drop_high_vif(X, threshold=8.0):\n",
        "    Xv = X.copy()\n",
        "    while True:\n",
        "        numeric = Xv.select_dtypes(include=[np.number])\n",
        "        if numeric.shape[1] <= 1:\n",
        "            break\n",
        "        vif = pd.Series([variance_inflation_factor(numeric.values, i) for i in range(numeric.shape[1])],\n",
        "                       index=numeric.columns)\n",
        "        max_vif = vif.max()\n",
        "        if max_vif > threshold:\n",
        "            Xv = Xv.drop(columns=[vif.idxmax()])\n",
        "        else:\n",
        "            break\n",
        "    return Xv\n",
        "\n",
        "print(\"\\n✅ Performing VIF pruning...\")\n",
        "X_train_reduced = drop_high_vif(X_train)\n",
        "kept_features = X_train_reduced.columns.tolist()\n",
        "X_test_reduced = X_test[kept_features]\n",
        "print(f\"✅ Features kept after VIF pruning: {len(kept_features)}\")\n",
        "\n",
        "kept_features = X_train_reduced.columns.tolist()\n",
        "print(\"\\n📌 Selected Features BEFORE Scaling:\")\n",
        "for f in kept_features:\n",
        "    print(\" -\", f)\n",
        "\n",
        "# --------------------------\n",
        "# 7. Scaling\n",
        "# --------------------------\n",
        "scaler = StandardScaler()\n",
        "X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_reduced), columns=kept_features)\n",
        "X_test_scaled = pd.DataFrame(scaler.transform(X_test_reduced), columns=kept_features)\n",
        "\n",
        "# --------------------------\n",
        "# 8. Train Models (IMPROVED)\n",
        "# --------------------------\n",
        "print(\"\\n✅ Training final models on full training set...\")\n",
        "\n",
        "# Random Forest (unchanged)\n",
        "rf = RandomForestRegressor(\n",
        "    n_estimators=800,\n",
        "    max_depth=None,         # unlimited depth\n",
        "    min_samples_split=2,\n",
        "    min_samples_leaf=1,\n",
        "    max_features=0.80,\n",
        "    bootstrap=True,\n",
        "    max_samples=1.0,\n",
        "    random_state=42,\n",
        "    n_jobs=-1\n",
        ")\n",
        "\n",
        "rf.fit(X_train_scaled, y_train)\n",
        "\n",
        "# Bagged Decision Tree (IMPROVED - replaces single Decision Tree)\n",
        "bdt = BaggingRegressor(\n",
        "    estimator=DecisionTreeRegressor(\n",
        "        max_depth=15,             # let trees grow deeper\n",
        "        min_samples_split=3,\n",
        "        min_samples_leaf=1,\n",
        "        random_state=42\n",
        "    ),\n",
        "    n_estimators=800,             # more trees = better averaging\n",
        "    max_samples=0.9,              # use more samples\n",
        "    max_features=1.0,             # allow more features → stronger trees\n",
        "    bootstrap=True,\n",
        "    random_state=42,\n",
        "    n_jobs=-1\n",
        ")\n",
        "\n",
        "bdt.fit(X_train_scaled, y_train)\n",
        "\n",
        "print(\"✅ Models trained successfully!\")\n",
        "\n",
        "# --------------------------\n",
        "# 9. Evaluate\n",
        "# --------------------------\n",
        "y_train_rf, y_test_rf = rf.predict(X_train_scaled), rf.predict(X_test_scaled)\n",
        "y_train_bdt, y_test_bdt = bdt.predict(X_train_scaled), bdt.predict(X_test_scaled)\n",
        "\n",
        "rf_train, rf_test = evaluate(y_train, y_train_rf), evaluate(y_test, y_test_rf)\n",
        "bdt_train, bdt_test = evaluate(y_train, y_train_bdt), evaluate(y_test, y_test_bdt)\n",
        "\n",
        "print(\"\\n✅ Random Forest Results:\")\n",
        "print_metrics(\"Train\", rf_train)\n",
        "print_metrics(\"Test \", rf_test)\n",
        "\n",
        "print(\"\\n✅ Bagged Decision Tree Results:\")\n",
        "print_metrics(\"Train\", bdt_train)\n",
        "print_metrics(\"Test \", bdt_test)\n",
        "\n",
        "# --------------------------\n",
        "# 10. Actual vs Predicted\n",
        "# --------------------------\n",
        "results_df = pd.DataFrame({\n",
        "    'Date': test_df['Date'].values,\n",
        "    'Actual': y_test.values,\n",
        "    'RF_Pred': y_test_rf,\n",
        "    'BDT_Pred': y_test_bdt\n",
        "})\n",
        "\n",
        "# Random Forest\n",
        "plt.figure(figsize=(10,4))\n",
        "plt.plot(results_df['Date'], results_df['Actual'], label='Actual', linewidth=1.2, alpha=0.8)\n",
        "plt.plot(results_df['Date'], results_df['RF_Pred'], label='Predicted (RF)', linewidth=1.2)\n",
        "plt.title(\"Random Forest: Actual vs Predicted Returns\")\n",
        "plt.xlabel(\"Date\"); plt.ylabel(\"Daily Return\")\n",
        "plt.legend(); plt.tight_layout(); plt.show()\n",
        "\n",
        "# Bagged Decision Tree\n",
        "plt.figure(figsize=(10,4))\n",
        "plt.plot(results_df['Date'], results_df['Actual'], label='Actual', linewidth=1.2, alpha=0.8)\n",
        "plt.plot(results_df['Date'], results_df['BDT_Pred'], label='Predicted (Bagged DT)', linewidth=1.2, color='orange')\n",
        "plt.title(\"Bagged Decision Tree: Actual vs Predicted Returns\")\n",
        "plt.xlabel(\"Date\"); plt.ylabel(\"Daily Return\")\n",
        "plt.legend(); plt.tight_layout(); plt.show()\n",
        "\n",
        "# --------------------------\n",
        "# 11. Feature Importance\n",
        "# --------------------------\n",
        "rf_imp = pd.Series(rf.feature_importances_, index=kept_features).sort_values(ascending=True)\n",
        "\n",
        "# For Bagged DT, calculate average importance across all trees, accounting for feature subsetting\n",
        "bdt_feature_importances = np.zeros(X_train_scaled.shape[1])\n",
        "for i, estimator in enumerate(bdt.estimators_):\n",
        "    bdt_feature_importances[bdt.estimators_features_[i]] += estimator.feature_importances_\n",
        "\n",
        "bdt_imp = pd.Series(bdt_feature_importances / len(bdt.estimators_), index=kept_features).sort_values(ascending=True)\n",
        "\n",
        "fig, ax = plt.subplots(1,2, figsize=(12,6))\n",
        "rf_imp.tail(15).plot(kind='barh', ax=ax[0], title='Random Forest Top Features', color='steelblue')\n",
        "bdt_imp.tail(15).plot(kind='barh', ax=ax[1], color='orange', title='Bagged Decision Tree Top Features')\n",
        "plt.tight_layout(); plt.show()\n",
        "\n",
        "# --------------------------\n",
        "# 12. Permutation Importance (RF)\n",
        "# --------------------------\n",
        "print(\"\\n✅ Computing permutation importance (this may take a moment)...\")\n",
        "perm = permutation_importance(rf, X_test_scaled, y_test, n_repeats=10, random_state=42, n_jobs=-1)\n",
        "perm_idx = np.argsort(perm.importances_mean)[::-1]\n",
        "\n",
        "plt.figure(figsize=(8,6))\n",
        "plt.barh(np.array(kept_features)[perm_idx][:15][::-1],\n",
        "         perm.importances_mean[perm_idx][:15][::-1],\n",
        "         color='steelblue')\n",
        "plt.title(\"Permutation Importance (Random Forest)\")\n",
        "plt.xlabel(\"Importance\")\n",
        "plt.tight_layout(); plt.show()\n",
        "\n",
        "# --------------------------\n",
        "# 13. SHAP Explainability (RF)\n",
        "# --------------------------\n",
        "print(\"\\n✅ Computing SHAP values (this may take a moment)...\")\n",
        "explainer = shap.TreeExplainer(rf)\n",
        "shap_values = explainer.shap_values(X_test_scaled)\n",
        "\n",
        "plt.title(\"SHAP Summary Plot (Random Forest)\")\n",
        "shap.summary_plot(shap_values, X_test_scaled, feature_names=kept_features)\n",
        "\n",
        "# --------------------------\n",
        "# 14. Model Comparison Table\n",
        "# --------------------------\n",
        "comparison = pd.DataFrame({\n",
        "    'Model': ['Random Forest', 'Bagged Decision Tree'],\n",
        "    'CV_Mean_R2': [np.mean(rf_cv_scores), np.mean(bdt_cv_scores)],\n",
        "    'CV_Std_R2': [np.std(rf_cv_scores), np.std(bdt_cv_scores)],\n",
        "    'Train_R2': [rf_train['R2'], bdt_train['R2']],\n",
        "    'Test_R2': [rf_test['R2'], bdt_test['R2']],\n",
        "    'Train_RMSE': [rf_train['RMSE'], bdt_train['RMSE']],\n",
        "    'Test_RMSE': [bdt_test['RMSE'], bdt_test['RMSE']] # Corrected: Use bdt_test for BDT RMSE\n",
        "})\n",
        "\n",
        "print(\"\\n\" + \"=\"*80)\n",
        "print(\"✅ COMPREHENSIVE MODEL COMPARISON:\")\n",
        "print(\"=\"*80)\n",
        "print(comparison.to_string(index=False))\n",
        "print(\"=\"*80)\n",
        "\n",
        "# Highlight CV performance per model\n",
        "rf_cv_range = (min(rf_cv_scores), max(rf_cv_scores))\n",
        "bdt_cv_range = (min(bdt_cv_scores), max(bdt_cv_scores))\n",
        "\n",
        "print(\"\\n📊 Cross-Validation Summary per Model:\")\n",
        "print(f\"Random Forest     - Mean R²: {np.mean(rf_cv_scores):.3f}, Range: {rf_cv_range}\")\n",
        "print(f\"Bagged Decision Tree - Mean R²: {np.mean(bdt_cv_scores):.3f}, Range: {bdt_cv_range}\")\n",
        "\n",
        "\n",
        "print(\"\\n🏆 Best Test Performance (Highest R²):\")\n",
        "best_test = comparison.loc[comparison['Test_R2'].idxmax(), 'Model']\n",
        "print(f\"   → {best_test}\")\n"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 1000
        },
        "id": "UpJCzFVcQSAv",
        "outputId": "ecdcb84f-6648-4c4f-8fe8-1ec4be3345ab"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "✅ Loaded dataset: (1277, 10)\n",
            "Date range: 2021-01-01 00:00:00 -> 2024-06-30 00:00:00\n",
            "\n",
            "✅ Missing values per column:\n",
            "Date                    0\n",
            "GBPAAUD_ExchangeRate    0\n",
            "Gold_Price              0\n",
            "Oil_Price               0\n",
            "GEPU_current            0\n",
            "UK                      0\n",
            "UK_Interest_Rate        0\n",
            "AU_Interest_Rate        0\n",
            "UK_Inflation            0\n",
            "AU_Inflation            0\n",
            "dtype: int64\n",
            "\n",
            "✅ Summary Statistics:\n"
          ]
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "                       count                 mean                  min  \\\n",
              "Date                    1277  2022-10-01 00:00:00  2021-01-01 00:00:00   \n",
              "GBPAAUD_ExchangeRate  1277.0              1.84124                1.653   \n",
              "Gold_Price            1277.0          1898.718089               1623.3   \n",
              "Oil_Price             1277.0             79.80119                47.62   \n",
              "GEPU_current          1277.0           227.018753             170.6739   \n",
              "UK                    1277.0           531.631857             168.6125   \n",
              "UK_Interest_Rate      1277.0             2.535161                  0.1   \n",
              "AU_Interest_Rate      1277.0             2.121535                  0.1   \n",
              "UK_Inflation          1277.0             5.392308             2.518371   \n",
              "AU_Inflation          1277.0             4.753717              2.86391   \n",
              "\n",
              "                                      25%                  50%  \\\n",
              "Date                  2021-11-16 00:00:00  2022-10-01 00:00:00   \n",
              "GBPAAUD_ExchangeRate               1.7798               1.8507   \n",
              "Gold_Price                         1786.9               1860.8   \n",
              "Oil_Price                           71.74                78.26   \n",
              "GEPU_current                     192.4917              221.266   \n",
              "UK                               394.1191             584.1855   \n",
              "UK_Interest_Rate                      0.1                 2.25   \n",
              "AU_Interest_Rate                      0.1                 2.35   \n",
              "UK_Inflation                     2.518371             6.793967   \n",
              "AU_Inflation                      2.86391             5.597015   \n",
              "\n",
              "                                      75%                  max         std  \n",
              "Date                  2023-08-16 00:00:00  2024-06-30 00:00:00         NaN  \n",
              "GBPAAUD_ExchangeRate               1.9024               1.9898    0.069959  \n",
              "Gold_Price                         1970.9               2433.9  163.311099  \n",
              "Oil_Price                           85.59                123.7   13.424009  \n",
              "GEPU_current                     253.3067             320.9541   41.613799  \n",
              "UK                               669.5651             844.8548  184.318316  \n",
              "UK_Interest_Rate                     5.25                 5.25     2.17651  \n",
              "AU_Interest_Rate                      4.1                 4.35    1.835604  \n",
              "UK_Inflation                     7.922049             7.922049    2.323085  \n",
              "AU_Inflation                     6.594097             6.594097    1.598566  "
            ],
            "text/html": [
              "\n",
              "  <div id=\"df-475f9484-d5fb-4ced-bb96-339c4b718766\" class=\"colab-df-container\">\n",
              "    <div>\n",
              "<style scoped>\n",
              "    .dataframe tbody tr th:only-of-type {\n",
              "        vertical-align: middle;\n",
              "    }\n",
              "\n",
              "    .dataframe tbody tr th {\n",
              "        vertical-align: top;\n",
              "    }\n",
              "\n",
              "    .dataframe thead th {\n",
              "        text-align: right;\n",
              "    }\n",
              "</style>\n",
              "<table border=\"1\" class=\"dataframe\">\n",
              "  <thead>\n",
              "    <tr style=\"text-align: right;\">\n",
              "      <th></th>\n",
              "      <th>count</th>\n",
              "      <th>mean</th>\n",
              "      <th>min</th>\n",
              "      <th>25%</th>\n",
              "      <th>50%</th>\n",
              "      <th>75%</th>\n",
              "      <th>max</th>\n",
              "      <th>std</th>\n",
              "    </tr>\n",
              "  </thead>\n",
              "  <tbody>\n",
              "    <tr>\n",
              "      <th>Date</th>\n",
              "      <td>1277</td>\n",
              "      <td>2022-10-01 00:00:00</td>\n",
              "      <td>2021-01-01 00:00:00</td>\n",
              "      <td>2021-11-16 00:00:00</td>\n",
              "      <td>2022-10-01 00:00:00</td>\n",
              "      <td>2023-08-16 00:00:00</td>\n",
              "      <td>2024-06-30 00:00:00</td>\n",
              "      <td>NaN</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>GBPAAUD_ExchangeRate</th>\n",
              "      <td>1277.0</td>\n",
              "      <td>1.84124</td>\n",
              "      <td>1.653</td>\n",
              "      <td>1.7798</td>\n",
              "      <td>1.8507</td>\n",
              "      <td>1.9024</td>\n",
              "      <td>1.9898</td>\n",
              "      <td>0.069959</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>Gold_Price</th>\n",
              "      <td>1277.0</td>\n",
              "      <td>1898.718089</td>\n",
              "      <td>1623.3</td>\n",
              "      <td>1786.9</td>\n",
              "      <td>1860.8</td>\n",
              "      <td>1970.9</td>\n",
              "      <td>2433.9</td>\n",
              "      <td>163.311099</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>Oil_Price</th>\n",
              "      <td>1277.0</td>\n",
              "      <td>79.80119</td>\n",
              "      <td>47.62</td>\n",
              "      <td>71.74</td>\n",
              "      <td>78.26</td>\n",
              "      <td>85.59</td>\n",
              "      <td>123.7</td>\n",
              "      <td>13.424009</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>GEPU_current</th>\n",
              "      <td>1277.0</td>\n",
              "      <td>227.018753</td>\n",
              "      <td>170.6739</td>\n",
              "      <td>192.4917</td>\n",
              "      <td>221.266</td>\n",
              "      <td>253.3067</td>\n",
              "      <td>320.9541</td>\n",
              "      <td>41.613799</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>UK</th>\n",
              "      <td>1277.0</td>\n",
              "      <td>531.631857</td>\n",
              "      <td>168.6125</td>\n",
              "      <td>394.1191</td>\n",
              "      <td>584.1855</td>\n",
              "      <td>669.5651</td>\n",
              "      <td>844.8548</td>\n",
              "      <td>184.318316</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>UK_Interest_Rate</th>\n",
              "      <td>1277.0</td>\n",
              "      <td>2.535161</td>\n",
              "      <td>0.1</td>\n",
              "      <td>0.1</td>\n",
              "      <td>2.25</td>\n",
              "      <td>5.25</td>\n",
              "      <td>5.25</td>\n",
              "      <td>2.17651</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>AU_Interest_Rate</th>\n",
              "      <td>1277.0</td>\n",
              "      <td>2.121535</td>\n",
              "      <td>0.1</td>\n",
              "      <td>0.1</td>\n",
              "      <td>2.35</td>\n",
              "      <td>4.1</td>\n",
              "      <td>4.35</td>\n",
              "      <td>1.835604</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>UK_Inflation</th>\n",
              "      <td>1277.0</td>\n",
              "      <td>5.392308</td>\n",
              "      <td>2.518371</td>\n",
              "      <td>2.518371</td>\n",
              "      <td>6.793967</td>\n",
              "      <td>7.922049</td>\n",
              "      <td>7.922049</td>\n",
              "      <td>2.323085</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>AU_Inflation</th>\n",
              "      <td>1277.0</td>\n",
              "      <td>4.753717</td>\n",
              "      <td>2.86391</td>\n",
              "      <td>2.86391</td>\n",
              "      <td>5.597015</td>\n",
              "      <td>6.594097</td>\n",
              "      <td>6.594097</td>\n",
              "      <td>1.598566</td>\n",
              "    </tr>\n",
              "  </tbody>\n",
              "</table>\n",
              "</div>\n",
              "    <div class=\"colab-df-buttons\">\n",
              "\n",
              "  <div class=\"colab-df-container\">\n",
              "    <button class=\"colab-df-convert\" onclick=\"convertToInteractive('df-475f9484-d5fb-4ced-bb96-339c4b718766')\"\n",
              "            title=\"Convert this dataframe to an interactive table.\"\n",
              "            style=\"display:none;\">\n",
              "\n",
              "  <svg xmlns=\"http://www.w3.org/2000/svg\" height=\"24px\" viewBox=\"0 -960 960 960\">\n",
              "    <path d=\"M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z\"/>\n",
              "  </svg>\n",
              "    </button>\n",
              "\n",
              "  <style>\n",
              "    .colab-df-container {\n",
              "      display:flex;\n",
              "      gap: 12px;\n",
              "    }\n",
              "\n",
              "    .colab-df-convert {\n",
              "      background-color: #E8F0FE;\n",
              "      border: none;\n",
              "      border-radius: 50%;\n",
              "      cursor: pointer;\n",
              "      display: none;\n",
              "      fill: #1967D2;\n",
              "      height: 32px;\n",
              "      padding: 0 0 0 0;\n",
              "      width: 32px;\n",
              "    }\n",
              "\n",
              "    .colab-df-convert:hover {\n",
              "      background-color: #E2EBFA;\n",
              "      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);\n",
              "      fill: #174EA6;\n",
              "    }\n",
              "\n",
              "    .colab-df-buttons div {\n",
              "      margin-bottom: 4px;\n",
              "    }\n",
              "\n",
              "    [theme=dark] .colab-df-convert {\n",
              "      background-color: #3B4455;\n",
              "      fill: #D2E3FC;\n",
              "    }\n",
              "\n",
              "    [theme=dark] .colab-df-convert:hover {\n",
              "      background-color: #434B5C;\n",
              "      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);\n",
              "      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));\n",
              "      fill: #FFFFFF;\n",
              "    }\n",
              "  </style>\n",
              "\n",
              "    <script>\n",
              "      const buttonEl =\n",
              "        document.querySelector('#df-475f9484-d5fb-4ced-bb96-339c4b718766 button.colab-df-convert');\n",
              "      buttonEl.style.display =\n",
              "        google.colab.kernel.accessAllowed ? 'block' : 'none';\n",
              "\n",
              "      async function convertToInteractive(key) {\n",
              "        const element = document.querySelector('#df-475f9484-d5fb-4ced-bb96-339c4b718766');\n",
              "        const dataTable =\n",
              "          await google.colab.kernel.invokeFunction('convertToInteractive',\n",
              "                                                    [key], {});\n",
              "        if (!dataTable) return;\n",
              "\n",
              "        const docLinkHtml = 'Like what you see? Visit the ' +\n",
              "          '<a target=\"_blank\" href=https://colab.research.google.com/notebooks/data_table.ipynb>data table notebook</a>'\n",
              "          + ' to learn more about interactive tables.';\n",
              "        element.innerHTML = '';\n",
              "        dataTable['output_type'] = 'display_data';\n",
              "        await google.colab.output.renderOutput(dataTable, element);\n",
              "        const docLink = document.createElement('div');\n",
              "        docLink.innerHTML = docLinkHtml;\n",
              "        element.appendChild(docLink);\n",
              "      }\n",
              "    </script>\n",
              "  </div>\n",
              "\n",
              "\n",
              "    <div id=\"df-a8e415eb-446d-4268-957c-a7c692d8c09f\">\n",
              "      <button class=\"colab-df-quickchart\" onclick=\"quickchart('df-a8e415eb-446d-4268-957c-a7c692d8c09f')\"\n",
              "                title=\"Suggest charts\"\n",
              "                style=\"display:none;\">\n",
              "\n",
              "<svg xmlns=\"http://www.w3.org/2000/svg\" height=\"24px\"viewBox=\"0 0 24 24\"\n",
              "     width=\"24px\">\n",
              "    <g>\n",
              "        <path d=\"M19 3H5c-1.1 0-2 .9-2 2v14c0 1.1.9 2 2 2h14c1.1 0 2-.9 2-2V5c0-1.1-.9-2-2-2zM9 17H7v-7h2v7zm4 0h-2V7h2v10zm4 0h-2v-4h2v4z\"/>\n",
              "    </g>\n",
              "</svg>\n",
              "      </button>\n",
              "\n",
              "<style>\n",
              "  .colab-df-quickchart {\n",
              "      --bg-color: #E8F0FE;\n",
              "      --fill-color: #1967D2;\n",
              "      --hover-bg-color: #E2EBFA;\n",
              "      --hover-fill-color: #174EA6;\n",
              "      --disabled-fill-color: #AAA;\n",
              "      --disabled-bg-color: #DDD;\n",
              "  }\n",
              "\n",
              "  [theme=dark] .colab-df-quickchart {\n",
              "      --bg-color: #3B4455;\n",
              "      --fill-color: #D2E3FC;\n",
              "      --hover-bg-color: #434B5C;\n",
              "      --hover-fill-color: #FFFFFF;\n",
              "      --disabled-bg-color: #3B4455;\n",
              "      --disabled-fill-color: #666;\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart {\n",
              "    background-color: var(--bg-color);\n",
              "    border: none;\n",
              "    border-radius: 50%;\n",
              "    cursor: pointer;\n",
              "    display: none;\n",
              "    fill: var(--fill-color);\n",
              "    height: 32px;\n",
              "    padding: 0;\n",
              "    width: 32px;\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart:hover {\n",
              "    background-color: var(--hover-bg-color);\n",
              "    box-shadow: 0 1px 2px rgba(60, 64, 67, 0.3), 0 1px 3px 1px rgba(60, 64, 67, 0.15);\n",
              "    fill: var(--button-hover-fill-color);\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart-complete:disabled,\n",
              "  .colab-df-quickchart-complete:disabled:hover {\n",
              "    background-color: var(--disabled-bg-color);\n",
              "    fill: var(--disabled-fill-color);\n",
              "    box-shadow: none;\n",
              "  }\n",
              "\n",
              "  .colab-df-spinner {\n",
              "    border: 2px solid var(--fill-color);\n",
              "    border-color: transparent;\n",
              "    border-bottom-color: var(--fill-color);\n",
              "    animation:\n",
              "      spin 1s steps(1) infinite;\n",
              "  }\n",
              "\n",
              "  @keyframes spin {\n",
              "    0% {\n",
              "      border-color: transparent;\n",
              "      border-bottom-color: var(--fill-color);\n",
              "      border-left-color: var(--fill-color);\n",
              "    }\n",
              "    20% {\n",
              "      border-color: transparent;\n",
              "      border-left-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "    }\n",
              "    30% {\n",
              "      border-color: transparent;\n",
              "      border-left-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "      border-right-color: var(--fill-color);\n",
              "    }\n",
              "    40% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "    }\n",
              "    60% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "    }\n",
              "    80% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "      border-bottom-color: var(--fill-color);\n",
              "    }\n",
              "    90% {\n",
              "      border-color: transparent;\n",
              "      border-bottom-color: var(--fill-color);\n",
              "    }\n",
              "  }\n",
              "</style>\n",
              "\n",
              "      <script>\n",
              "        async function quickchart(key) {\n",
              "          const quickchartButtonEl =\n",
              "            document.querySelector('#' + key + ' button');\n",
              "          quickchartButtonEl.disabled = true;  // To prevent multiple clicks.\n",
              "          quickchartButtonEl.classList.add('colab-df-spinner');\n",
              "          try {\n",
              "            const charts = await google.colab.kernel.invokeFunction(\n",
              "                'suggestCharts', [key], {});\n",
              "          } catch (error) {\n",
              "            console.error('Error during call to suggestCharts:', error);\n",
              "          }\n",
              "          quickchartButtonEl.classList.remove('colab-df-spinner');\n",
              "          quickchartButtonEl.classList.add('colab-df-quickchart-complete');\n",
              "        }\n",
              "        (() => {\n",
              "          let quickchartButtonEl =\n",
              "            document.querySelector('#df-a8e415eb-446d-4268-957c-a7c692d8c09f button');\n",
              "          quickchartButtonEl.style.display =\n",
              "            google.colab.kernel.accessAllowed ? 'block' : 'none';\n",
              "        })();\n",
              "      </script>\n",
              "    </div>\n",
              "\n",
              "    </div>\n",
              "  </div>\n"
            ],
            "application/vnd.google.colaboratory.intrinsic+json": {
              "type": "dataframe",
              "summary": "{\n  \"name\": \"print(f\\\"   \\u2192 {best_test}\\\")\",\n  \"rows\": 10,\n  \"fields\": [\n    {\n      \"column\": \"count\",\n      \"properties\": {\n        \"dtype\": \"date\",\n        \"min\": \"1277\",\n        \"max\": \"1277\",\n        \"num_unique_values\": 1,\n        \"samples\": [\n          \"1277\"\n        ],\n        \"semantic_type\": \"\",\n        \"description\": \"\"\n      }\n    },\n    {\n      \"column\": \"mean\",\n      \"properties\": {\n        \"dtype\": \"date\",\n        \"min\": \"1970-01-01 00:00:00.000000001\",\n        \"max\": \"2022-10-01 00:00:00\",\n        \"num_unique_values\": 10,\n        \"samples\": [\n          5.392308176194205\n        ],\n        \"semantic_type\": \"\",\n        \"description\": \"\"\n      }\n    },\n    {\n      \"column\": \"min\",\n      \"properties\": {\n        \"dtype\": \"date\",\n        \"min\": \"1970-01-01 00:00:00\",\n        \"max\": \"2021-01-01 00:00:00\",\n        \"num_unique_values\": 9,\n        \"samples\": [\n          2.518371\n        ],\n        \"semantic_type\": \"\",\n        \"description\": \"\"\n      }\n    },\n    {\n      \"column\": \"25%\",\n      \"properties\": {\n        \"dtype\": \"date\",\n        \"min\": \"1970-01-01 00:00:00\",\n        \"max\": \"2021-11-16 00:00:00\",\n        \"num_unique_values\": 9,\n        \"samples\": [\n          2.518371\n        ],\n        \"semantic_type\": \"\",\n        \"description\": \"\"\n      }\n    },\n    {\n      \"column\": \"50%\",\n      \"properties\": {\n        \"dtype\": \"date\",\n        \"min\": \"1970-01-01 00:00:00.000000001\",\n        \"max\": \"2022-10-01 00:00:00\",\n        \"num_unique_values\": 10,\n        \"samples\": [\n          6.793967\n        ],\n        \"semantic_type\": \"\",\n        \"description\": \"\"\n      }\n    },\n    {\n      \"column\": \"75%\",\n      \"properties\": {\n        \"dtype\": \"date\",\n        \"min\": \"1970-01-01 00:00:00.000000001\",\n        \"max\": \"2023-08-16 00:00:00\",\n        \"num_unique_values\": 10,\n        \"samples\": [\n          7.922049\n        ],\n        \"semantic_type\": \"\",\n        \"description\": \"\"\n      }\n    },\n    {\n      \"column\": \"max\",\n      \"properties\": {\n        \"dtype\": \"date\",\n        \"min\": \"1970-01-01 00:00:00.000000001\",\n        \"max\": \"2024-06-30 00:00:00\",\n        \"num_unique_values\": 10,\n        \"samples\": [\n          7.922049\n        ],\n        \"semantic_type\": \"\",\n        \"description\": \"\"\n      }\n    },\n    {\n      \"column\": \"std\",\n      \"properties\": {\n        \"dtype\": \"date\",\n        \"min\": 0.069959353352981,\n        \"max\": 184.31831618208665,\n        \"num_unique_values\": 9,\n        \"samples\": [\n          2.3230845970148835\n        ],\n        \"semantic_type\": \"\",\n        \"description\": \"\"\n      }\n    }\n  ]\n}"
            }
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 1000x400 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 600x400 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 1200x500 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 1200x500 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 1200x500 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 1200x500 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\n",
            "✅ After feature engineering: (1267, 22)\n",
            "\n",
            "✅ Train samples: 1013, Test samples: 254\n",
            "\n",
            "✅ Performing Time Series Cross-Validation...\n",
            "Cross-validation progress:\n",
            "  Fold 1: RF R²=0.5122, Bagged DT R²=0.5094\n",
            "  Fold 2: RF R²=0.5570, Bagged DT R²=0.5888\n",
            "  Fold 3: RF R²=0.5794, Bagged DT R²=0.6105\n",
            "  Fold 4: RF R²=0.7287, Bagged DT R²=0.7618\n",
            "  Fold 5: RF R²=0.7818, Bagged DT R²=0.8008\n",
            "\n",
            "✅ Cross-Validation Results:\n",
            "Random Forest     - Mean R²: 0.6318 (+/- 0.1044)\n",
            "Bagged Dec. Tree  - Mean R²: 0.6543 (+/- 0.1098)\n"
          ]
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 1000x500 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\n",
            "✅ Performing VIF pruning...\n",
            "✅ Features kept after VIF pruning: 11\n",
            "\n",
            "📌 Selected Features BEFORE Scaling:\n",
            " - UK\n",
            " - AU_Interest_Rate\n",
            " - Gold_Price_Daily_Return\n",
            " - Oil_Price_Daily_Return\n",
            " - ExchangeRate_Return_Lag1\n",
            " - ExchangeRate_Return_Lag2\n",
            " - ExchangeRate_Return_Lag3\n",
            " - ExchangeRate_MA3\n",
            " - ExchangeRate_Vol3\n",
            " - ExchangeRate_MA5\n",
            " - ExchangeRate_MA10\n",
            "\n",
            "✅ Training final models on full training set...\n",
            "✅ Models trained successfully!\n",
            "\n",
            "✅ Random Forest Results:\n",
            "Train: RMSE=0.000789, MAE=0.000434, R2=0.9642\n",
            "Test : RMSE=0.001159, MAE=0.000712, R2=0.8620\n",
            "\n",
            "✅ Bagged Decision Tree Results:\n",
            "Train: RMSE=0.000857, MAE=0.000451, R2=0.9578\n",
            "Test : RMSE=0.001118, MAE=0.000665, R2=0.8716\n"
          ]
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 1000x400 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 1000x400 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 1200x600 with 2 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\n",
            "✅ Computing permutation importance (this may take a moment)...\n"
          ]
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 800x600 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\n",
            "✅ Computing SHAP values (this may take a moment)...\n"
          ]
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 800x590 with 2 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\n",
            "================================================================================\n",
            "✅ COMPREHENSIVE MODEL COMPARISON:\n",
            "================================================================================\n",
            "               Model  CV_Mean_R2  CV_Std_R2  Train_R2  Test_R2  Train_RMSE  Test_RMSE\n",
            "       Random Forest    0.631809   0.104433  0.964237 0.861989    0.000789   0.001118\n",
            "Bagged Decision Tree    0.654251   0.109755  0.957847 0.871553    0.000857   0.001118\n",
            "================================================================================\n",
            "\n",
            "📊 Cross-Validation Summary per Model:\n",
            "Random Forest     - Mean R²: 0.632, Range: (0.5122160778947066, 0.7818081863586825)\n",
            "Bagged Decision Tree - Mean R²: 0.654, Range: (0.5093815099284458, 0.8008345425379099)\n",
            "\n",
            "🏆 Best Test Performance (Highest R²):\n",
            "   → Bagged Decision Tree\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "!pip install lime\n"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "MRlpz8ccdBIo",
        "outputId": "96da3ba6-12ad-4037-ad32-97af3342c462"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Collecting lime\n",
            "  Downloading lime-0.2.0.1.tar.gz (275 kB)\n",
            "\u001b[?25l     \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m0.0/275.7 kB\u001b[0m \u001b[31m?\u001b[0m eta \u001b[36m-:--:--\u001b[0m\r\u001b[2K     \u001b[91m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m\u001b[91m╸\u001b[0m\u001b[90m━\u001b[0m \u001b[32m266.2/275.7 kB\u001b[0m \u001b[31m11.6 MB/s\u001b[0m eta \u001b[36m0:00:01\u001b[0m\r\u001b[2K     \u001b[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\u001b[0m \u001b[32m275.7/275.7 kB\u001b[0m \u001b[31m7.2 MB/s\u001b[0m eta \u001b[36m0:00:00\u001b[0m\n",
            "\u001b[?25h  Preparing metadata (setup.py) ... \u001b[?25l\u001b[?25hdone\n",
            "Requirement already satisfied: matplotlib in /usr/local/lib/python3.12/dist-packages (from lime) (3.10.0)\n",
            "Requirement already satisfied: numpy in /usr/local/lib/python3.12/dist-packages (from lime) (2.0.2)\n",
            "Requirement already satisfied: scipy in /usr/local/lib/python3.12/dist-packages (from lime) (1.16.3)\n",
            "Requirement already satisfied: tqdm in /usr/local/lib/python3.12/dist-packages (from lime) (4.67.1)\n",
            "Requirement already satisfied: scikit-learn>=0.18 in /usr/local/lib/python3.12/dist-packages (from lime) (1.6.1)\n",
            "Requirement already satisfied: scikit-image>=0.12 in /usr/local/lib/python3.12/dist-packages (from lime) (0.25.2)\n",
            "Requirement already satisfied: networkx>=3.0 in /usr/local/lib/python3.12/dist-packages (from scikit-image>=0.12->lime) (3.5)\n",
            "Requirement already satisfied: pillow>=10.1 in /usr/local/lib/python3.12/dist-packages (from scikit-image>=0.12->lime) (11.3.0)\n",
            "Requirement already satisfied: imageio!=2.35.0,>=2.33 in /usr/local/lib/python3.12/dist-packages (from scikit-image>=0.12->lime) (2.37.2)\n",
            "Requirement already satisfied: tifffile>=2022.8.12 in /usr/local/lib/python3.12/dist-packages (from scikit-image>=0.12->lime) (2025.10.16)\n",
            "Requirement already satisfied: packaging>=21 in /usr/local/lib/python3.12/dist-packages (from scikit-image>=0.12->lime) (25.0)\n",
            "Requirement already satisfied: lazy-loader>=0.4 in /usr/local/lib/python3.12/dist-packages (from scikit-image>=0.12->lime) (0.4)\n",
            "Requirement already satisfied: joblib>=1.2.0 in /usr/local/lib/python3.12/dist-packages (from scikit-learn>=0.18->lime) (1.5.2)\n",
            "Requirement already satisfied: threadpoolctl>=3.1.0 in /usr/local/lib/python3.12/dist-packages (from scikit-learn>=0.18->lime) (3.6.0)\n",
            "Requirement already satisfied: contourpy>=1.0.1 in /usr/local/lib/python3.12/dist-packages (from matplotlib->lime) (1.3.3)\n",
            "Requirement already satisfied: cycler>=0.10 in /usr/local/lib/python3.12/dist-packages (from matplotlib->lime) (0.12.1)\n",
            "Requirement already satisfied: fonttools>=4.22.0 in /usr/local/lib/python3.12/dist-packages (from matplotlib->lime) (4.60.1)\n",
            "Requirement already satisfied: kiwisolver>=1.3.1 in /usr/local/lib/python3.12/dist-packages (from matplotlib->lime) (1.4.9)\n",
            "Requirement already satisfied: pyparsing>=2.3.1 in /usr/local/lib/python3.12/dist-packages (from matplotlib->lime) (3.2.5)\n",
            "Requirement already satisfied: python-dateutil>=2.7 in /usr/local/lib/python3.12/dist-packages (from matplotlib->lime) (2.9.0.post0)\n",
            "Requirement already satisfied: six>=1.5 in /usr/local/lib/python3.12/dist-packages (from python-dateutil>=2.7->matplotlib->lime) (1.17.0)\n",
            "Building wheels for collected packages: lime\n",
            "  Building wheel for lime (setup.py) ... \u001b[?25l\u001b[?25hdone\n",
            "  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=f79856909c8d7d3ab8199f8b66922e3f36bd30c303a49bcfb00f1af84cfcd589\n",
            "  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a\n",
            "Successfully built lime\n",
            "Installing collected packages: lime\n",
            "Successfully installed lime-0.2.0.1\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "from lime.lime_tabular import LimeTabularExplainer\n",
        "\n",
        "# Create LIME explainer\n",
        "explainer_lime = LimeTabularExplainer(\n",
        "    training_data=X_train_scaled.values,\n",
        "    feature_names=kept_features,\n",
        "    mode='regression'\n",
        ")\n",
        "\n",
        "# Select a test instance to explain (e.g., first instance)\n",
        "i = 0\n",
        "\n",
        "# LIME explanation for Random Forest\n",
        "lime_exp_rf = explainer_lime.explain_instance(\n",
        "    X_test_scaled.values[i],\n",
        "    rf.predict,\n",
        "    num_features=10\n",
        ")\n",
        "\n",
        "print(\"\\n✅ LIME Explanation (Random Forest) for test instance 0:\")\n",
        "for feature, effect in lime_exp_rf.as_list():\n",
        "    print(f\"{feature}: {effect:.6f}\")\n",
        "\n",
        "# LIME explanation for Bagged Decision Tree\n",
        "lime_exp_bdt = explainer_lime.explain_instance(\n",
        "    X_test_scaled.values[i],\n",
        "    bdt.predict,\n",
        "    num_features=10\n",
        ")\n",
        "\n",
        "print(\"\\n✅ LIME Explanation (Bagged Decision Tree) for test instance 0:\")\n",
        "for feature, effect in lime_exp_bdt.as_list():\n",
        "    print(f\"{feature}: {effect:.6f}\")\n"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "y0fEOK96djZ8",
        "outputId": "68fbb8c2-e5c9-4cb1-82b6-b61a227684ed"
      },
      "execution_count": null,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "\n",
            "✅ LIME Explanation (Random Forest) for test instance 0:\n",
            "0.02 < ExchangeRate_MA3 <= 0.55: 0.001851\n",
            "-0.02 < ExchangeRate_Return_Lag2 <= 0.36: -0.001126\n",
            "-0.02 < ExchangeRate_Return_Lag1 <= 0.36: -0.000331\n",
            "0.07 < UK <= 0.84: 0.000271\n",
            "-0.29 < Oil_Price_Daily_Return <= -0.04: 0.000210\n",
            "-0.31 < Gold_Price_Daily_Return <= -0.01: 0.000135\n",
            "-0.63 < ExchangeRate_MA5 <= 0.03: -0.000118\n",
            "ExchangeRate_Vol3 <= -0.70: -0.000072\n",
            "AU_Interest_Rate > 1.07: 0.000023\n",
            "0.03 < ExchangeRate_MA10 <= 0.66: 0.000020\n",
            "\n",
            "✅ LIME Explanation (Bagged Decision Tree) for test instance 0:\n",
            "0.02 < ExchangeRate_MA3 <= 0.55: 0.002076\n",
            "-0.02 < ExchangeRate_Return_Lag2 <= 0.36: -0.000762\n",
            "-0.02 < ExchangeRate_Return_Lag1 <= 0.36: -0.000464\n",
            "-0.31 < Gold_Price_Daily_Return <= -0.01: 0.000334\n",
            "AU_Interest_Rate > 1.07: 0.000170\n",
            "ExchangeRate_Vol3 <= -0.70: -0.000141\n",
            "-0.63 < ExchangeRate_MA5 <= 0.03: -0.000118\n",
            "0.03 < ExchangeRate_MA10 <= 0.66: 0.000114\n",
            "ExchangeRate_Return_Lag3 <= -0.34: 0.000082\n",
            "-0.29 < Oil_Price_Daily_Return <= -0.04: -0.000029\n"
          ]
        }
      ]
    }
  ]
}
